In [ ]:
# For prerequisites running the following sample, visit https://help.aliyun.com/document_detail/611472.html

import dashscope
from dashscope import Generation
from http import HTTPStatus
import json

dashscope.api_key='<your-dashscope-api-key>'

response=Generation.call(
    model='qwen-v1',
    prompt='就当前的海洋污染的情况，写一份限塑的倡议书提纲，需要有理有据地号召大家克制地使用塑料制品'
    )

if response.status_code==HTTPStatus.OK:
    print(json.dumps(response.output, indent=4, ensure_ascii=False))
else:
    print('Code: %d, status: %s, message: %s' % (response.status_code, response.code, response.message))

In [ ]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import json
import dashscope
from dashscope import Generation
from http import HTTPStatus

dashscope.api_key = 'your-dashscope-api-key'

class RequestHandler(BaseHTTPRequestHandler):
    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        post_data = self.rfile.read(content_length)
        request = json.loads(post_data.decode('utf-8'))
        
        response = Generation.call(
            model='qwen-v1',
            prompt=request['prompt']
        )

        self.send_response(HTTPStatus.OK)
        self.send_header('Content-type', 'application/json')
        self.end_headers()
        
        if response.status_code == HTTPStatus.OK:
            output = {
                'output': response.output,
                'usage': response.usage
            }
            self.wfile.write(json.dumps(output).encode('utf-8'))
        else:
            error = {
                'code': response.code,
                'message': response.message
            }
            self.wfile.write(json.dumps(error).encode('utf-8'))

def run_server():
    server_address = ('', 8000)
    httpd = HTTPServer(server_address, RequestHandler)
    print('Starting server on port 8000...')
    httpd.serve_forever()

run_server()